In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from pandas import DataFrame, Index, Series
from torch.utils.data import Dataset as TorchDataset

import tsdm
from tsdm.datasets import TimeSeriesCollection
from tsdm.random.samplers import HierarchicalSampler, SlidingWindowSampler

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)

In [ ]:
ds = tsdm.datasets.KIWI_RUNS()

In [ ]:
ds.timeseries

In [ ]:
ds.timeseries

In [ ]:
ds.time_features.dtypes

In [ ]:
from tsdm.datasets import KIWI

In [ ]:
ds = KIWI()
ds

In [ ]:
ds.index

In [ ]:
ds[439, 15325]

In [ ]:
TSC = TimeSeriesCollection(
    index=ds.index,
    timeseries=ds.timeseries,
    metadata=ds.metadata,
    time_features=ds.time_features,
    value_features=ds.value_features,
    metadata_features=ds.metadata_features,
)

## Sampler

In [ ]:
ts = ds.timeseries.loc[(439, 15325)]

In [ ]:
sampler = SlidingWindowSampler(ts.index, horizons=["2h", "1h"], stride="1h")

In [ ]:
next(iter(sampler))

## Construct the Sampler

In [ ]:
TSC

In [ ]:
from torch.utils.data import RandomSampler, SubsetRandomSampler

In [ ]:
next(iter(SubsetRandomSampler(TSC.index)))

In [ ]:
TSC[(439, 15325)]

In [ ]:
subsamplers = {
    key: SlidingWindowSampler(ds.timeseries.index, horizons=["2h", "1h"], stride="1h")
    for key, ds in TSC.items()
}
sampler = HierarchicalSampler(TSC, subsamplers, shuffle=False)

In [ ]:
outer_key, (forecasting_horizon, prediction_horizon) = next(iter(sampler))

In [ ]:
TSC[outer_key][forecasting_horizon]

# TimeSeriesCollectionForecastingTask

## TimeSeriesCollection Sample

In [ ]:
from typing import NamedTuple

from tsdm.utils.strings import *


class Inputs(NamedTuple):
    """tuple of inputs"""

    t: int
    x: int
    t_target: int
    metadata: int

    def __repr__(self):
        return repr_namedtuple(self, recursive=False)


class Sample(NamedTuple):
    """A sample for forecasting task"""

    key: int
    inputs: Inputs
    targets: int
    # originals: int

    def __repr__(self):
        return repr_namedtuple(self, recursive=False)

In [ ]:
from dataclasses import dataclass


@dataclass
class TimeSeriesCollectionForecastingTask(TorchDataset):
    r"""Creates sample from a TimeSeriesCollection."""

    dataset: TimeSeriesCollection
    targets: Index
    controls: Index
    observables: Index

    def __getitem__(self, key) -> Sample:
        assert isinstance(key, tuple) and len(key) == 2
        outer_key, inner_key = key
        assert isinstance(inner_key, list) and len(inner_key) == 2
        observation_horizon, forecasting_horizon = inner_key

        tsd = self.dataset[outer_key]
        md = tsd.metadata

        obs = tsd[observation_horizon]
        pre = tsd[forecasting_horizon]
        horizon = observation_horizon | forecasting_horizon
        ts = tsd[horizon]

        return Sample(
            key=outer_key,
            inputs=Inputs(ts, obs, ts, pre),
            targets=pre,
        )

In [ ]:
key = next(iter(sampler))

In [ ]:
task = TimeSeriesCollectionForecastingTask(
    TSC, targets=None, controls=None, observables=None
)
task[key]

# Mapping Dataset

In [ ]:
d = dict(enumerate("asdfghjkl"))

In [ ]:
from collections.abc import Mapping
from dataclasses import dataclass

from torch.utils.data import Dataset

In [ ]:
@dataclass
class MyMapping(Dataset, Mapping):
    internal_dict: dict

    def __iter__(self):
        return iter(self.internal_dict)

    def __getitem__(self, key):
        return self.internal_dict[key]

    def __len__(self):
        return len(self.internal_dict)

In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(MyMapping(d))

In [ ]:
from typing import Any, Callable, Sequence


def flatten_dict(
    d: dict[str, Any],
    /,
    *,
    recursive: bool = True,
    join_fn: Callable[[Sequence[str]], str] = ".".join,
) -> dict[str, Any]:
    r"""Flatten dictionaries recursively."""
    result = {}
    for key, item in d.items():
        if isinstance(item, dict) and recursive:
            subdict = flatten_dict(item, recursive=True, join_fn=join_fn)
            for subkey, subitem in subdict.items():
                result[join_fn((key, subkey))] = subitem
        else:
            result[key] = item
    return result


def unflatten_dict(
    d: dict[str, Any],
    /,
    *,
    recursive: bool = True,
    split_fn: Callable[[str], Sequence[str]] = lambda s: s.split(".", maxsplit=1),
) -> dict[str, Any]:
    r"""Unflatten dictionaries recursively."""
    result = {}
    for key, item in d.items():
        split = split_fn(key)
        result.setdefault(split[0], {})
        if len(split) > 1 and recursive:
            assert len(split) == 2
            subdict = unflatten_dict(
                {split[1]: item}, recursive=recursive, split_fn=split_fn
            )
            result[split[0]] |= subdict
        else:
            result[split[0]] = item
    return result


a = {
    "a": True,
    "b": 42,
    "c": "foo",
    "foo": {"a": 1, "b": 2, "c": 3},
    "bar": {"a": {"a": 1}, "b": {"a": 1}, "c": {"a": 1}},
    "baz": {"a": True, "b": 2},  # <-- does not work!
}

print(flat := flatten_dict(a))
print(unflat := unflatten_dict(a))
assert unflat == a

In [ ]:
flat

In [ ]:
unflatten_dict(flat)

In [ ]:
"a".split(".")

In [ ]:
dir(".")